In [5]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from PIL import Image, ImageTk
from datetime import datetime
import os

# Load your models and scaler
data = pd.read_csv('diabetes.csv')
data.columns = data.columns.str.strip()  # Strip spaces

# Prepare features and target variable
X = data.drop('Outcome(Diabetes)', axis=1)
y = data['Outcome(Diabetes)']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train models
logistic_model = LogisticRegression()
logistic_model.fit(X_train_scaled, y_train)

tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)

# Calculate accuracy for both models
logistic_accuracy = accuracy_score(y_test, logistic_model.predict(X_test_scaled))
tree_accuracy = accuracy_score(y_test, tree_model.predict(X_test))


def calculate_age(dob):
    today = datetime.now()
    birth_date = datetime.strptime(dob, '%Y-%m-%d')
    age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    return age


def validate_inputs():
    error_messages = []
    # Validate glucose
    try:
        glucose = float(glucose_entry.get())
        if not (0 <= glucose <= 200):
            error_messages.append("Glucose must be between 0 and 200.")
    except ValueError:
        error_messages.append("Glucose must be a number.")
    
    # Validate blood pressure
    try:
        blood_pressure = float(blood_pressure_entry.get())
        if not (0 <= blood_pressure <= 200):
            error_messages.append("Blood Pressure must be between 0 and 200.")
    except ValueError:
        error_messages.append("Blood Pressure must be a number.")
    
    # Validate skin thickness
    try:
        skin_thickness = float(skin_thickness_entry.get())
        if not (0 <= skin_thickness <= 100):
            error_messages.append("Skin Thickness must be between 0 and 100.")
    except ValueError:
        error_messages.append("Skin Thickness must be a number.")
    
    # Validate insulin
    try:
        insulin = float(insulin_entry.get())
        if not (0 <= insulin <= 1000):
            error_messages.append("Insulin must be between 0 and 1000.")
    except ValueError:
        error_messages.append("Insulin must be a number.")
    
    # Validate weight
    try:
        weight = float(weight_entry.get())
        if weight <= 0 or weight > 500:
            error_messages.append("Weight must be between 0 and 500 kg.")
    except ValueError:
        error_messages.append("Weight must be a number.")
    
    # Validate height
    try:
        height = float(height_entry.get())
        if height <= 0 or height > 300:
            error_messages.append("Height must be between 0 and 300 cm.")
    except ValueError:
        error_messages.append("Height must be a number.")
    
    # Validate diabetes pedigree function
    try:
        pedigree_function = float(pedigree_function_entry.get())
        if not (0 <= pedigree_function <= 2):
            error_messages.append("Diabetes Pedigree Function must be between 0 and 2.")
    except ValueError:
        error_messages.append("Diabetes Pedigree Function must be a number.")
    
    # Validate date of birth
    dob = dob_entry.get()
    try:
        datetime.strptime(dob, '%Y-%m-%d')
        age = calculate_age(dob)
    except ValueError:
        error_messages.append("Date of Birth must be in YYYY-MM-DD format.")
    
    # Validate pregnancies
    try:
        pregnancies = int(pregnancies_entry.get())
        if pregnancies < 0:
            error_messages.append("Pregnancies cannot be negative.")
    except ValueError:
        error_messages.append("Pregnancies must be a whole number.")
    
    return error_messages


def display_errors(errors):
    for error_label in error_labels:
        error_label.config(text="")
    
    for i, error in enumerate(errors):
        if i < len(error_labels):
            error_labels[i].config(text=error)


def predict_diabetes(model, scaler, glucose, blood_pressure, skin_thickness, insulin, bmi, pedigree_function, age, pregnancies):
    input_data = pd.DataFrame([[glucose, blood_pressure, skin_thickness, insulin, bmi, pedigree_function, age, pregnancies]],
                               columns=X.columns)
    
    if scaler:
        input_data_scaled = scaler.transform(input_data)
    else:
        input_data_scaled = input_data
        
    prediction = model.predict(input_data_scaled)
    return 'Diabetic' if prediction[0] == 1 else 'Not Diabetic'


def generate_recommendations(glucose, blood_pressure, bmi):
    recommendations = []
    if glucose > 140:
        recommendations.append("Your glucose level is high. Consider reducing sugar intake and consult a dietitian.")
    if blood_pressure > 130:
        recommendations.append("Your blood pressure is a bit high. Regular exercise and a low-sodium diet can help. Consider seeing a doctor.")
    if bmi >= 25:
        recommendations.append("You are overweight. Consider a balanced diet and regular physical activity to manage your weight.")
    if glucose <= 140 and blood_pressure <= 130 and bmi < 25:
        recommendations.append("Your readings are normal! Maintain a healthy lifestyle.")

    recommendations.append("Always consult a healthcare provider for personalized advice.")
    return "\n".join(recommendations)


def update_accuracy_label(*args):
    selected_algorithm = algorithm_var.get()
    if selected_algorithm == "Logistic Regression":
        accuracy_label.config(text=f"Accuracy: {logistic_accuracy:.2%}")
    else:
        accuracy_label.config(text=f"Accuracy: {tree_accuracy:.2%}")


def display_prediction_window(user_input_summary, recommendations):
    result_window = tk.Toplevel()
    result_window.title("Prediction Results")
    result_window.configure(bg='#f7f7f7')

    result_frame = tk.Frame(result_window, bg='#f7f7f7', padx=20, pady=20)
    result_frame.pack(padx=10, pady=10)

    # Title
    title_label = tk.Label(result_frame, text="Diabetes Prediction Report", font=('Helvetica', 16, 'bold'), bg='#f7f7f7')
    title_label.pack(pady=(0, 10))

    # User Input Summary
    input_frame = tk.Frame(result_frame, bg='#f7f7f7')
    input_frame.pack(fill='x', pady=(0, 10))

    user_summary_lines = user_input_summary.split("\n")
    for line in user_summary_lines:
        summary_label = tk.Label(input_frame, text=line, font=('Helvetica', 12), bg='#f7f7f7', justify=tk.LEFT)
        summary_label.pack(anchor='w')

    # Recommendations
    rec_frame = tk.Frame(result_frame, bg='#f7f7f7')
    rec_frame.pack(fill='x', pady=(0, 10))
    
    recommendations_label = tk.Label(rec_frame, text="Recommendations:", font=('Helvetica', 14), bg='#f7f7f7')
    recommendations_label.pack(anchor='w')
    
    recommendations_lines = recommendations.split("\n")
    for line in recommendations_lines:
        rec_label = tk.Label(rec_frame, text=line, font=('Helvetica', 12), bg='#f7f7f7', justify=tk.LEFT)
        rec_label.pack(anchor='w')

    # Prediction Result
    result_label = tk.Label(result_frame, text=f"Prediction: {user_summary_lines[-1].split(': ')[-1]}", font=('Helvetica', 16, 'bold'), bg='#f7f7f7', fg='#d9534f')
    result_label.pack(pady=(10, 0))

    # Print and Close buttons
    button_frame = tk.Frame(result_frame, bg='#f7f7f7')
    button_frame.pack(pady=(10, 0))

    print_button = tk.Button(button_frame, text="Print Report", command=lambda: print_report(user_input_summary, recommendations), font=('Helvetica', 12), bg='#5bc0de', fg='white')
    print_button.pack(side=tk.LEFT, padx=5)

    close_button = tk.Button(button_frame, text="Close", command=result_window.destroy, font=('Helvetica', 12), bg='#5cb85c', fg='white')
    close_button.pack(side=tk.LEFT, padx=5)


def print_report(user_input_summary, recommendations):
    report_text = f"{user_input_summary}\n\n{recommendations}"
    try:
        # Save report to a temporary file
        with open("diabetes_report.txt", "w") as report_file:
            report_file.write(report_text)

        # Print the report using the default text editor
        os.startfile("diabetes_report.txt", "print")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while trying to print the report: {e}")


def on_predict():
    errors = validate_inputs()
    
    if errors:
        display_errors(errors)
        return

    glucose = float(glucose_entry.get())
    blood_pressure = float(blood_pressure_entry.get())
    skin_thickness = float(skin_thickness_entry.get())
    insulin = float(insulin_entry.get())
    weight = float(weight_entry.get())
    height = float(height_entry.get())
    bmi = weight / (height / 100) ** 2
    pedigree_function = float(pedigree_function_entry.get())
    dob = dob_entry.get()
    age = calculate_age(dob)
    pregnancies = int(pregnancies_entry.get())

    selected_algorithm = algorithm_var.get()
    
    if selected_algorithm == "Logistic Regression":
        result = predict_diabetes(logistic_model, scaler, glucose, blood_pressure, skin_thickness, insulin, bmi, pedigree_function, age, pregnancies)
    else:
        result = predict_diabetes(tree_model, None, glucose, blood_pressure, skin_thickness, insulin, bmi, pedigree_function, age, pregnancies)

    user_input_summary = (
        f"Glucose: {glucose}\n"
        f"Blood Pressure: {blood_pressure}\n"
        f"Skin Thickness: {skin_thickness}\n"
        f"Insulin: {insulin}\n"
        f"Weight: {weight} kg\n"
        f"Height: {height} cm\n"
        f"BMI: {bmi:.2f}\n"
        f"Diabetes Pedigree Function: {pedigree_function}\n"
        f"Age: {age} years\n"
        f"Pregnancies: {pregnancies}\n"
        f"Prediction: {result}"
    )
    
    recommendations = generate_recommendations(glucose, blood_pressure, bmi)
    display_prediction_window(user_input_summary, recommendations)

class SplashScreen:
    def __init__(self, root):
        self.root = root
        self.root.overrideredirect(True)
        width, height = 500, 300
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        x = (screen_width // 2) - (width // 2)
        y = (screen_height // 2) - (height // 2)
        self.root.geometry(f'{width}x{height}+{x}+{y}')
        
        image = Image.open("Diabetes (Flash screen).jpg")
        image = image.resize((width, height - 80), Image.LANCZOS)
        self.bg_image = ImageTk.PhotoImage(image)
        
        label = tk.Label(self.root, image=self.bg_image)
        label.pack()

        button_frame = tk.Frame(self.root)
        button_frame.pack(side=tk.BOTTOM, pady=10)

        acknowledge_button = tk.Button(button_frame, text="Acknowledge", font=('Helvetica', 14), command=self.open_main_window)
        acknowledge_button.pack(side=tk.LEFT, padx=5)

        exit_button = tk.Button(button_frame, text="Exit", font=('Helvetica', 14), command=self.exit_program)
        exit_button.pack(side=tk.LEFT, padx=5)

    def open_main_window(self):
        self.root.destroy()
        create_main_window()

    def exit_program(self):
        self.root.destroy()

def create_main_window():
    root = tk.Tk()
    root.title("Diabetes Prediction")
    root.configure(bg='#f7f7f7')

    padx, pady = 10, 10

    global error_labels
    error_labels = []

    def create_entry_with_error(label_text, row):
        label = tk.Label(root, text=label_text, font=('Helvetica', 12), bg='#f7f7f7')
        label.grid(row=row, column=0, padx=padx, pady=pady)
        entry = tk.Entry(root, font=('Helvetica', 12), bd=2)
        entry.grid(row=row, column=1, padx=padx, pady=pady)
        error_label = tk.Label(root, text="", font=('Helvetica', 10), fg='red', bg='#f7f7f7')
        error_label.grid(row=row, column=2, padx=(padx, 0), pady=pady)
        error_labels.append(error_label)
        return entry

    global glucose_entry
    glucose_entry = create_entry_with_error("Glucose:", 0)
    global blood_pressure_entry
    blood_pressure_entry = create_entry_with_error("Blood Pressure:", 1)
    global skin_thickness_entry
    skin_thickness_entry = create_entry_with_error("Skin Thickness:", 2)
    global insulin_entry
    insulin_entry = create_entry_with_error("Insulin:", 3)
    global weight_entry
    weight_entry = create_entry_with_error("Weight (kg):", 4)
    global height_entry
    height_entry = create_entry_with_error("Height (cm):", 5)
    global pedigree_function_entry
    pedigree_function_entry = create_entry_with_error("Diabetes Pedigree Function:", 6)
    global dob_entry
    dob_entry = create_entry_with_error("Date of Birth (YYYY-MM-DD):", 7)
    global pregnancies_entry
    pregnancies_entry = create_entry_with_error("Pregnancies:", 8)

    tk.Label(root, text="Select Algorithm:", font=('Helvetica', 12), bg='#f7f7f7').grid(row=9, column=0, padx=padx, pady=pady)
    global algorithm_var
    algorithm_var = tk.StringVar(value="Logistic Regression")
    algorithm_menu = tk.OptionMenu(root, algorithm_var, "Logistic Regression", "Decision Tree", command=update_accuracy_label)
    algorithm_menu.grid(row=9, column=1, padx=padx, pady=pady)

    global accuracy_label
    accuracy_label = tk.Label(root, text=f"Accuracy: {logistic_accuracy:.2%}", font=('Helvetica', 12), bg='#f7f7f7')
    accuracy_label.grid(row=9, column=2, padx=padx, pady=pady)

    predict_button = tk.Button(root, text="Predict", font=('Helvetica', 12, 'bold'), bg='#5cb85c', fg='white', bd=4, command=on_predict)
    predict_button.grid(row=10, columnspan=3, pady=20)

    root.mainloop()


if __name__ == "__main__":
    splash_root = tk.Tk()
    splash = SplashScreen(splash_root)
    splash_root.mainloop()


